In [8]:
import torch
import torch.nn as nn 
import pandas as pd 
import numpy as np
import mlflow
import pickle

In [10]:
mlflow.set_tracking_uri('file:/home/naradaw/dev/Charles_Schwab/code/RnD/v2/mlflow_experiments')

mlflow.set_experiment("/portfolio-contructor-v2")

<Experiment: artifact_location='file:///home/naradaw/dev/Charles_Schwab/code/RnD/v2/mlflow_experiments/930648686917041142', creation_time=1730715551772, experiment_id='930648686917041142', last_update_time=1730715551772, lifecycle_stage='active', name='/portfolio-contructor-v2', tags={}>

In [2]:
with open("/home/naradaw/dev/Charles_Schwab/data/symbol_universe/snp_unique_100_2019", "rb") as fp:
    symbol_universe = pickle.load(fp)
    
symbol_universe = symbol_universe[:20]

In [3]:
data_path = data_path = "/home/naradaw/dev/Charles_Schwab/data/w_features/v1/2024_10_31/dataset_sqs_2024_10_31_11_18.pkl"

with open(data_path, 'rb') as f:
    data = pickle.load(f)

In [4]:
#shape from (symbols, time_steps, seq_len, features) -> (time_steps ,seq_len, symbols, features)

data_features = np.array([data[symbol]['features'] for symbol in symbol_universe]).transpose(1,2,0,3)
data_returns = np.array([data[symbol]['returns'] for symbol in symbol_universe]).transpose(1,0)

In [5]:
del data

In [ ]:
data_features[0].shape

(60, 20, 87)

In [20]:
sample_ten = torch.Tensor(data_features[0]).to('cuda')


In [27]:
logged_model = 'runs:/7af8fd7e0c154de0af41e4b932a7134e/portfolio_constructor_2024_11_05_10_10'

# Load model as a PyFuncModel.
loaded_model = mlflow.pytorch.load_model(logged_model)

In [32]:
srem = loaded_model.SREM
srem

SREM(
  (compressor): Linear(in_features=87, out_features=88, bias=True)
  (TE): TransformerEncoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=88, out_features=88, bias=True)
        )
        (linear1): Linear(in_features=88, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=88, bias=True)
        (norm1): LayerNorm((88,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((88,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
)

In [29]:
caan = loaded_model.CAAN
caan

CAAN(
  (W_Q): Linear(in_features=88, out_features=15, bias=True)
  (W_K): Linear(in_features=88, out_features=15, bias=True)
  (W_V): Linear(in_features=88, out_features=15, bias=True)
  (W_O): Linear(in_features=15, out_features=88, bias=True)
  (normalizer): LayerNorm((15,), eps=1e-05, elementwise_affine=True)
  (scorer): Linear(in_features=15, out_features=1, bias=True)
)

In [30]:
srem_out = srem(sample_ten)
srem_out

tensor([[-0.4363,  0.4076, -1.0288,  ..., -0.2631,  0.7371, -0.1746],
        [-0.4369,  0.4108, -1.0169,  ..., -0.2368,  0.7053, -0.1776],
        [-0.4120,  0.3909, -1.0362,  ..., -0.2780,  0.7479, -0.1593],
        ...,
        [-0.4247,  0.4047, -1.0180,  ..., -0.2414,  0.7050, -0.1666],
        [-0.4213,  0.3959, -1.0334,  ..., -0.2722,  0.7426, -0.1676],
        [-0.4179,  0.3925, -1.0228,  ..., -0.3036,  0.7723, -0.1651]],
       device='cuda:0', grad_fn=<MeanBackward1>)

In [31]:
caan_out = caan(srem_out)
caan_out

tensor([-5.0502, -5.0502, -5.0502, -5.0506, -5.0502, -5.0506, -5.0505, -5.0503,
        -5.0501, -5.0503, -5.0505, -5.0507, -5.0501, -5.0503, -5.0502, -5.0502,
        -5.0502, -5.0502, -5.0502, -5.0505], device='cuda:0',
       grad_fn=<SqueezeBackward0>)